In [12]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

# Patch the Kiwi search tool schema so model knows required fields
for t in tools:
    if t.name == "search_flights":
        t.args_schema['properties']['adults'] = {"type": "integer", "default": 1}
        t.args_schema['required'].append("adults")


In [18]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    "gpt-5-nano",
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel agent. No follow up questions."
)

In [20]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct one way flight from San Francisco to Tokyo on March 31st.")]},
    config
    )

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_fkvnfDrSIlv4BlM07bGDTq18", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}

In [ ]:
from pprint import pprint

pprint(response)

NameError: name 'response' is not defined

In [ ]:
print(response["messages"][-1].content)